# Assignment 3

In this assigment, we will work with the *Forest Fire* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/162/forest+fires). Extract the data files into the subdirectory: `../data/fires/` (relative to `./src/`).

## Objective

+ The model objective is to predict the area affected by forest fires given the features set. 
+ The objective of this exercise is to assess your ability to construct and evaluate model pipelines.
+ Please note: the instructions are not meant to be 100% prescriptive, but instead they are a set of minimum requirements. If you find predictive performance gains by applying additional steps, by all means show them. 

## Variable Description

From the description file contained in the archive (`forestfires.names`), we obtain the following variable descriptions:

1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. month - month of the year: "jan" to "dec" 
4. day - day of the week: "mon" to "sun"
5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
6. DMC - DMC index from the FWI system: 1.1 to 291.3 
7. DC - DC index from the FWI system: 7.9 to 860.6 
8. ISI - ISI index from the FWI system: 0.0 to 56.10
9. temp - temperature in Celsius degrees: 2.2 to 33.30
10. RH - relative humidity in %: 15.0 to 100
11. wind - wind speed in km/h: 0.40 to 9.40 
12. rain - outside rain in mm/m2 : 0.0 to 6.4 
13. area - the burned area of the forest (in ha): 0.00 to 1090.84 









### Specific Tasks

+ Construct four model pipelines, out of combinations of the following components:

    + Preprocessors:

        - A simple processor that only scales numeric variables and recodes categorical variables.
        - A transformation preprocessor that scales numeric variables and applies a non-linear transformation.
    
    + Regressor:

        - A baseline regressor, which could be a [K-nearest neighbours model](https://open.spotify.com/track/4R3AU2pjv8ge2siX1fVbZs?si=b2712f32da0e4358) or a simple [linear regression model](https://scikit-learn.org/stable/modules/linear_model.html)
        - An advanced regressor of your choice (e.g., Random Forest, Neural Network, etc.)

+ Evaluate tune and evaluate each of the four model pipelines. 

    - Select a [performance metric](https://scikit-learn.org/stable/modules/linear_model.html) out of the following options: explained variance, max error, root mean squared error (RMSE), mean absolute error (MAE), r-squared.
    - *TIPS*: 
    
        * Out of the suggested metrics above, [some are correlation metrics, but this is a prediction problem](https://www.tmwr.org/performance#performance). Choose wisely (and don't choose the incorrect options.) 

+ Select the best-performing model and explain its predictions.

    - Provide local explanations.
    - Obtain global explanations and recommend a variable selection strategy.

+ Export your model as a pickle file.


You can work on the Jupyter notebook, as this experiment is fairly short (no need to use sacred). 

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../data/adult/`, then you can use the code below to load them.

In [1]:
import pandas as pd
columns = [
    'coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'area' 
]
fires_dt = (pd.read_csv('../05_src/data/fires/forestfires.csv', header = None, names = columns))


In [2]:
fires_dt = fires_dt.drop(0)
fires_dt.head()

,coord_x,coord_y,month,day,ffmc,dmc,dc,isi,temp,rh,wind,rain,area
1,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0,0
2,7,4,oct,tue,90.6,35.4,669.1,6.7,18,33,0.9,0,0
3,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0,0
4,8,6,mar,fri,91.7,33.3,77.5,9,8.3,97,4,0.2,0
5,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0,0


In [3]:
################################################################
# data treatment
################################################################
import numpy as np
fires_dt_adjusted = pd.DataFrame()

# 'coord_x', 'coord_y','ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind'
fires_dt_adjusted['month'] = fires_dt['month']
fires_dt_adjusted['day'] = fires_dt['day']
fires_dt_adjusted['coord_x'] = pd.to_numeric ( fires_dt['coord_x'], errors = 'coerce')
fires_dt_adjusted['coord_y'] = pd.to_numeric ( fires_dt['coord_y'], errors = 'coerce')
fires_dt_adjusted['ffmc'] = pd.to_numeric ( fires_dt['ffmc'], errors = 'coerce')
fires_dt_adjusted['dmc'] = pd.to_numeric ( fires_dt['dmc'], errors = 'coerce')
fires_dt_adjusted['dc'] = pd.to_numeric ( fires_dt['dc'], errors = 'coerce')
fires_dt_adjusted['isi'] = pd.to_numeric ( fires_dt['isi'], errors = 'coerce')
fires_dt_adjusted['temp'] = pd.to_numeric ( fires_dt['temp'], errors = 'coerce')
fires_dt_adjusted['rh'] = pd.to_numeric ( fires_dt['rh'], errors = 'coerce')
fires_dt_adjusted['wind'] = pd.to_numeric ( fires_dt['wind'], errors = 'coerce')
fires_dt_adjusted['rain'] = pd.to_numeric(fires_dt['rain'], errors='coerce')
fires_dt_adjusted['area'] = pd.to_numeric ( fires_dt['area'], errors = 'coerce')

fires_dt_adjusted = fires_dt_adjusted.dropna(axis=0)


In [4]:
print ("fires_dt.shape" , fires_dt.shape)

print ("fires_dt_adjusted.shape" , fires_dt_adjusted.shape)

fires_dt.shape (517, 13)
fires_dt_adjusted.shape (517, 13)


In [5]:
analysys_df = pd.DataFrame()

analysys_df['varaibles'] = fires_dt.columns
analysys_df['dtype'] = np.array(fires_dt.dtypes)
analysys_df['is_null'] = np.array(fires_dt.isnull().sum())

analysys_df

,varaibles,dtype,is_null
0,coord_x,object,0
1,coord_y,object,0
2,month,object,0
3,day,object,0
4,ffmc,object,0
5,dmc,object,0
6,dc,object,0
7,isi,object,0
8,temp,object,0
9,rh,object,0


In [6]:
# data overview
analysys_df2 = pd.DataFrame()

analysys_df2['varaibles'] = fires_dt_adjusted.columns
analysys_df2['dtype'] = np.array(fires_dt_adjusted.dtypes)
analysys_df2['is_null'] = np.array(fires_dt_adjusted.isnull().sum())

analysys_df2

,varaibles,dtype,is_null
0,month,object,0
1,day,object,0
2,coord_x,int64,0
3,coord_y,int64,0
4,ffmc,float64,0
5,dmc,float64,0
6,dc,float64,0
7,isi,float64,0
8,temp,float64,0
9,rh,int64,0


# Get X and Y

Create the features data frame and target data.

In [7]:
from sklearn.model_selection import train_test_split

# The model objective is to predict the area affected by forest fires given the features set. 
x_df = fires_dt_adjusted.drop (columns=['area'], axis=1)

y_df = fires_dt_adjusted['area']

X_train, X_test, Y_train, Y_test = train_test_split(x_df, y_df, test_size = 0.3, random_state = 42)

# Preprocessing

Create two [Column Transformers](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), called preproc1 and preproc2, with the following guidelines:

- Numerical variables

    * (Preproc 1 and 2) Scaling: use a scaling method of your choice (Standard, Robust, Min-Max). 
    * Preproc 2 only: 
        
        + Choose a transformation for any of your input variables (or several of them). Evaluate if this transformation is convenient.
        + The choice of scaler is up to you.

- Categorical variables: 
    
    * (Preproc 1 and 2) Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) where appropriate.


+ The only difference between preproc1 and preproc2 is the non-linear transformation of the numerical variables.
    


### Preproc 1

Create preproc1 below.

+ Numeric: scaled variables, no other transforms.
+ Categorical: one-hot encoding.

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, PowerTransformer, RobustScaler, OneHotEncoder
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.model_selection import  cross_validate
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier


In [61]:
# preproc1
num_columns = [ 'coord_x', 'coord_y',  'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain']
non_num_columns = ['month', 'day']

# numerical Variables
pipe_num_columns = Pipeline([
    ('standardizer', RobustScaler())
])

# NON numerical Variables
pipe_non_num_columns = Pipeline([
   ('standardizer', OneHotEncoder(handle_unknown='infrequent_if_exist') )
])

ctransform_simple = ColumnTransformer([
    ('numeric_simple', pipe_num_columns, num_columns),
    ('non_numeric_simple', pipe_non_num_columns, non_num_columns)]
    , remainder='drop'
    )


### Preproc 2

Create preproc1 below.

+ Numeric: scaled variables, non-linear transformation to one or more variables.
+ Categorical: one-hot encoding.

In [62]:
# preproc2
from sklearn.preprocessing import StandardScaler, FunctionTransformer

num_columns = [ 'coord_x', 'coord_y',  'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind']
num_column_rain = ['rain']
non_num_columns = ['month', 'day']

# Custom transformer for non-linear transformation the log
def apply_log(x):
    return np.log1p(x)  # Apply log transformation (log(1 + x))

log_transformer = FunctionTransformer(func=apply_log, validate=False)


# numerical Variables
pipe2_num_columns = Pipeline([
    ('standardizer', StandardScaler())
])

pipe2_num_column_rain = Pipeline([
    ('standardizer', StandardScaler(), num_column_rain),
    ('log_rain', log_transformer, ['rain'] )
])


# NON numerical Variables
pipe2_non_num_columns = Pipeline([
   ('standardizer', OneHotEncoder(handle_unknown='infrequent_if_exist') )
])

ctransform_simple2 = ColumnTransformer([
    ('numeric_simple', pipe2_num_columns, num_columns),
    ('log_rain' , pipe2_num_column_rain , num_column_rain),
    ('non_numeric_simple', pipe2_non_num_columns, non_num_columns)]
    , remainder='drop'
    )


## Model Pipeline


Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `regressor` and assign a regression model to it. 

## Regressor

+ Use a regression model to perform a prediction. 

    - Choose a baseline regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Choose a more advance regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Both model choices are up to you, feel free to experiment.

In [58]:
# Pipeline A = preproc1 + baseline
from sklearn.linear_model import LogisticRegression, LinearRegression

pipe_A = Pipeline([
    ('preprocessing', ctransform_simple),
    ('clf', LogisticRegression())
])

pipe_A


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('numeric_simple',
                                                  Pipeline(steps=[('standardizer',
                                                                   RobustScaler())]),
                                                  ['coord_x', 'coord_y', 'ffmc',
                                                   'dmc', 'dc', 'isi', 'temp',
                                                   'rh', 'wind', 'rain'])])),
                ('clf', LogisticRegression())])

In [12]:
# Pipeline B = preproc2 + baseline

pipe_B = Pipeline([
    ('preprocessing', ctransform_simple2),
    ('clf', LinearRegression())
])

pipe_B

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('numeric_simple',
                                                  Pipeline(steps=[('standardizer',
                                                                   StandardScaler())]),
                                                  ['coord_x', 'coord_y', 'ffmc',
                                                   'dmc', 'dc', 'isi', 'temp',
                                                   'rh', 'wind']),
                                                 ('log_rain',
                                                  Pipeline(steps=[('standardizer',
                                                                   StandardScaler(),
                                                                   ['rain']),
                                                                  ('log_rain',
                                                                   FunctionTransformer(func=<function apply_log at 0x000002227C771D30>),
                                                                   ['rain'])]),
                                                  ['rain']),
                                                 ('non_numeric_simple',
                                                  Pipeline(steps=[('standardizer',
                                                                   OneHotEncoder(handle_unknown='infrequent_if_exist'))]),
                                                  ['month', 'day'])])),
                ('clf', LinearRegression())])

In [13]:
# Pipeline C = preproc1 + advanced model
from sklearn.ensemble import RandomForestRegressor

pipe_C = Pipeline([
    ('preprocessing', ctransform_simple),
    ('clf', RandomForestRegressor())
])

pipe_C



Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('numeric_simple',
                                                  Pipeline(steps=[('standardizer',
                                                                   RobustScaler())]),
                                                  ['coord_x', 'coord_y', 'ffmc',
                                                   'dmc', 'dc', 'isi', 'temp',
                                                   'rh', 'wind', 'rain']),
                                                 ('non_numeric_simple',
                                                  Pipeline(steps=[('standardizer',
                                                                   OneHotEncoder(handle_unknown='infrequent_if_exist'))]),
                                                  ['month', 'day'])])),
                ('clf', RandomForestRegressor())])

In [14]:
# Pipeline D = preproc2 + advanced model

pipe_D = Pipeline([
    ('preprocessing', ctransform_simple2),
    ('clf', RandomForestRegressor())
])

pipe_D


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('numeric_simple',
                                                  Pipeline(steps=[('standardizer',
                                                                   StandardScaler())]),
                                                  ['coord_x', 'coord_y', 'ffmc',
                                                   'dmc', 'dc', 'isi', 'temp',
                                                   'rh', 'wind']),
                                                 ('log_rain',
                                                  Pipeline(steps=[('standardizer',
                                                                   StandardScaler(),
                                                                   ['rain']),
                                                                  ('log_rain',
                                                                   FunctionTransformer(func=<function apply_log at 0x000002227C771D30>),
                                                                   ['rain'])]),
                                                  ['rain']),
                                                 ('non_numeric_simple',
                                                  Pipeline(steps=[('standardizer',
                                                                   OneHotEncoder(handle_unknown='infrequent_if_exist'))]),
                                                  ['month', 'day'])])),
                ('clf', RandomForestRegressor())])

# Tune Hyperparams

+ Perform GridSearch on each of the four pipelines. 
+ Tune at least one hyperparameter per pipeline.
+ Experiment with at least four value combinations per pipeline.

In [59]:
pipe_A.get_params()

{'memory': None,
 'steps': [('preprocessing',
   ColumnTransformer(transformers=[('numeric_simple',
                                    Pipeline(steps=[('standardizer',
                                                     RobustScaler())]),
                                    ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc',
                                     'isi', 'temp', 'rh', 'wind', 'rain'])])),
  ('clf', LogisticRegression())],
 'verbose': False,
 'preprocessing': ColumnTransformer(transformers=[('numeric_simple',
                                  Pipeline(steps=[('standardizer',
                                                   RobustScaler())]),
                                  ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc',
                                   'isi', 'temp', 'rh', 'wind', 'rain'])]),
 'clf': LogisticRegression(),
 'preprocessing__force_int_remainder_cols': True,
 'preprocessing__n_jobs': None,
 'preprocessing__remainder': 'drop',
 'preprocessing__sparse_threshold': 0.

In [60]:
# Pipeline A = preproc1 + baseline
from sklearn.model_selection import GridSearchCV

scoring = ['neg_log_loss', 'accuracy', 'precision', 'recall'] 

# The clf__tol -> parameter controls the tolerance for stopping the optimization process.
# The clf__solver -> parameter specifies the optimization algorithm used by certain models

# Parameters for Logistic Regression
param_grid_Logistic_Regression = {
    'clf__solver': ['lbfgs','newton-cg', 'sag', 'saga'],
    'clf__tol': [0.0001]
}

# Parfameters for Linear Regression()
param_grid_Linear_Regression = {
    'clf__fit_intercept': [True, False],
    'clf__copy_X': [True,False]
}

if np.array(pipe_A['clf']).astype(str) == 'LogisticRegression()':
    param_grid = param_grid_Logistic_Regression
elif np.array(pipe_A['clf']).astype(str) == 'LinearRegression()':
    param_grid = param_grid_Linear_Regression


grid_cv_A = GridSearchCV( # GridSearchCV is a brute force approach for testing all param_grid combinations
    estimator  = pipe_A, 
    param_grid = param_grid, 
    scoring    = scoring, 
    cv         = 5,
    refit = "neg_log_loss", # once you finish testing all, reutrn the best model according to Log Loss
    return_train_score=True) 

grid_cv_A.fit(X_train , Y_train)



ValueError: 
All the 20 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\klaes\miniconda3\envs\scale2prod\lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\klaes\miniconda3\envs\scale2prod\lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "c:\Users\klaes\miniconda3\envs\scale2prod\lib\site-packages\sklearn\pipeline.py", line 476, in fit
    self._final_estimator.fit(Xt, y, **last_step_params["fit"])
  File "c:\Users\klaes\miniconda3\envs\scale2prod\lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(e

In [43]:
pipe_B.get_params()

{'memory': None,
 'steps': [('preprocessing',
   ColumnTransformer(transformers=[('numeric_simple',
                                    Pipeline(steps=[('standardizer',
                                                     StandardScaler())]),
                                    ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc',
                                     'isi', 'temp', 'rh', 'wind']),
                                   ('log_rain',
                                    Pipeline(steps=[('standardizer',
                                                     StandardScaler(), ['rain']),
                                                    ('log_rain',
                                                     FunctionTransformer(func=<function apply_log at 0x000002227C771D30>),
                                                     ['rain'])]),
                                    ['rain']),
                                   ('non_numeric_simple',
                                    Pipeline(steps=

In [52]:
# Evaluate baseline model
# Pipeline B = preproc2 + baseline
from sklearn.model_selection import GridSearchCV

scoring = ['neg_log_loss', 'accuracy', 'precision', 'recall'] 

# The clf__tol -> parameter controls the tolerance for stopping the optimization process.
# The clf__solver -> parameter specifies the optimization algorithm used by certain models

# Parameters for Logistic Regression
param_grid_Logistic_Regression = {
    'clf__solver': ['lbfgs','newton-cg', 'sag', 'saga'],
    'clf__tol': [0.0001]
}

# Parfameters for Linear Regression()
param_grid_Linear_Regression = {
    'clf__fit_intercept': [True, False],
    'clf__copy_X': [True,False]
}

if np.array(pipe_A['clf']).astype(str) == 'LogisticRegression()':
    param_grid = param_grid_Logistic_Regression
elif np.array(pipe_A['clf']).astype(str) == 'LinearRegression()':
    param_grid = param_grid_Linear_Regression


grid_cv_A = GridSearchCV( # GridSearchCV is a brute force approach for testing all param_grid combinations
    estimator  = pipe_B, 
    param_grid = param_grid, 
    scoring    = scoring, 
    cv         = 5,
    refit = "neg_log_loss",
    return_train_score=True) # once you finish testing all, reutrn the best model according to Log Loss

grid_cv_A.fit(X_train , Y_train)



ValueError: 
All the 20 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\klaes\miniconda3\envs\scale2prod\lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\klaes\miniconda3\envs\scale2prod\lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "c:\Users\klaes\miniconda3\envs\scale2prod\lib\site-packages\sklearn\pipeline.py", line 472, in fit
    Xt = self._fit(X, y, routed_params)
  File "c:\Users\klaes\miniconda3\envs\scale2prod\lib\site-packages\sklearn\pipeline.py", line 409, in _fit
    X, fitted_transformer = fit_transform_one_c

In [44]:
pipe_C.get_params()

{'memory': None,
 'steps': [('preprocessing',
   ColumnTransformer(transformers=[('numeric_simple',
                                    Pipeline(steps=[('standardizer',
                                                     RobustScaler())]),
                                    ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc',
                                     'isi', 'temp', 'rh', 'wind', 'rain']),
                                   ('non_numeric_simple',
                                    Pipeline(steps=[('standardizer',
                                                     OneHotEncoder(handle_unknown='infrequent_if_exist'))]),
                                    ['month', 'day'])])),
  ('clf', RandomForestRegressor())],
 'verbose': False,
 'preprocessing': ColumnTransformer(transformers=[('numeric_simple',
                                  Pipeline(steps=[('standardizer',
                                                   RobustScaler())]),
                                  ['coord_x', '

In [50]:
# Pipeline C = preproc1 + advanced model

scoring = ['neg_log_loss', 'accuracy', 'precision', 'recall'] 

# The clf__tol -> parameter controls the tolerance for stopping the optimization process.
# The clf__solver -> parameter specifies the optimization algorithm used by certain models

param_grid = {
    'clf__bootstrap': [True , False] # smaller value means string regulatization ... 1.0 means not regularization at all
    }

grid_cv_B = GridSearchCV( # GridSearchCV is a brute force approach for testing all param_grid combinations
    estimator  = pipe_C, 
    param_grid = param_grid, 
    scoring    = scoring, 
    cv         = 5,
    refit = "neg_log_loss",
    return_train_score=True) # once you finish testing all, reutrn the best model according to Log Loss

grid_cv_B.fit(X_train , Y_train)


AttributeError: Pipeline has none of the following attributes: predict_proba.

In [51]:
# Pipeline D = preproc2 + advanced model
scoring = ['neg_log_loss', 'accuracy', 'precision', 'recall'] 

# The clf__tol -> parameter controls the tolerance for stopping the optimization process.
# The clf__solver -> parameter specifies the optimization algorithm used by certain models

param_grid = {
    'clf__bootstrap': [True , False] # smaller value means string regulatization ... 1.0 means not regularization at all
    }

grid_cv_B = GridSearchCV( # GridSearchCV is a brute force approach for testing all param_grid combinations
    estimator  = pipe_D, 
    param_grid = param_grid, 
    scoring    = scoring, 
    cv         = 5,
    refit = "neg_log_loss",
    return_train_score=True) # once you finish testing all, reutrn the best model according to Log Loss

grid_cv_B.fit(X_train , Y_train)

ValueError: 
All the 10 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\klaes\miniconda3\envs\scale2prod\lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\klaes\miniconda3\envs\scale2prod\lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "c:\Users\klaes\miniconda3\envs\scale2prod\lib\site-packages\sklearn\pipeline.py", line 472, in fit
    Xt = self._fit(X, y, routed_params)
  File "c:\Users\klaes\miniconda3\envs\scale2prod\lib\site-packages\sklearn\pipeline.py", line 409, in _fit
    X, fitted_transformer = fit_transform_one_c

# Evaluate

+ Which model has the best performance?

# Export

+ Save the best performing model to a pickle file.

# Explain

+ Use SHAP values to explain the following only for the best-performing model:

    - Select an observation in your test set and explain which are the most important features that explain that observation's specific prediction.

    - In general, across the complete training set, which features are the most and least important.

+ If you were to remove features from the model, which ones would you remove? Why? How would you test that these features are actually enhancing model performance?

*(Answer here.)*

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-3`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_3.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Cortez,Paulo and Morais,Anbal. (2008). Forest Fires. UCI Machine Learning Repository. https://doi.org/10.24432/C5D88D.